In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

pd.options.display.float_format = '{:.0f}'.format # Supress scientific notation

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip -nv

with ZipFile('book-crossings.zip', 'r') as zObject:
    zObject.extractall()

os.remove('book-crossings.zip')

filenames = ['BX-Book-Ratings.csv', 'BX-Books.csv', 'BX-Users.csv']

2024-01-11 23:26:15 URL:https://cdn.freecodecamp.org/project-data/books/book-crossings.zip [26085508/26085508] -> "book-crossings.zip" [1]


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    filenames[1],
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    filenames[0],
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

for file in filenames:
    os.remove(file)

In [18]:
print(f"Books: {df_books.shape}  -  Ratings: {df_ratings.shape}")

Books: (271379, 3)  -  Ratings: (1149780, 3)


In [23]:
df_ratings.groupby('isbn').count()

,user,rating
isbn,,
0330299891,2,2
0375404120,2,2
0586045007,1,1
9022906116,2,2
9032803328,1,1
...,...,...
cn113107,1,1
ooo7156103,1,1
§423350229,1,1


In [25]:
df_ratings.groupby('user').count()

,isbn,rating
user,,
2,1,1
7,1,1
8,18,18
9,3,3
10,2,2
...,...,...
278846,2,2
278849,4,4
278851,23,23


In [49]:
user_list = df_ratings.groupby('user').count()
user_filter = user_list.loc[user_list.isbn >= 200].index.to_list()

isbn_list = df_ratings.groupby('isbn').count()
isbn_filter = isbn_list.loc[isbn_list['user'] >= 100].index.to_list()

print(f'Users with more than 200 reviews: {len(user_filter)}  -  Books with more than 200 reviews: {len(isbn_filter)}')

Users with more than 200 reviews: 905  -  Books with more than 200 reviews: 731


In [51]:
df_filter = df_ratings.loc[(df_ratings.isbn.isin(isbn_filter)) & (df_ratings.user.isin(user_filter))]

In [59]:
df = df_filter.merge(right=df_books, on='isbn').drop_duplicates(subset=['title', 'user'])

df.sample(10)

,user,isbn,rating,title,author
15429,178667,0452264464,5,Beloved (Plume Contemporary Fiction),Toni Morrison
34337,148744,0060922532,0,Pigs in Heaven,Barbara Kingsolver
20437,230249,0446609404,0,Cradle and All,James Patterson
22877,15408,0060928336,0,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells
48646,236283,0743211383,10,Dreamcatcher,Stephen King
43281,60244,0449221512,0,I Is for Innocent,Sue Grafton
27976,11993,068484267X,0,Angela's Ashes: A Memoir,Frank McCourt
21353,245645,0440213290,0,The Copper Beech,Maeve Binchy
4909,177090,0515132020,8,Heaven and Earth (Three Sisters Island Trilogy),Nora Roberts
10194,110973,0312974256,9,Welcome to Temptation,Jennifer Crusie


In [74]:
matrix = pd.pivot(df, index='title', columns='user', values='rating').fillna(0)
matrix.sample(5)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
Seinlanguage,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Fahrenheit 451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
You Belong To Me,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Smilla's Sense of Snow,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Simple Abundance: A Daybook of Comfort and Joy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
N_RECOMMENDS = 5

In [77]:
neighbors = NearestNeighbors(algorithm='auto', n_neighbors=N_RECOMMENDS + 1)
neighbors.fit(matrix)

NearestNeighbors(n_neighbors=6)

In [78]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  list = [book, []]
  
  if book in df['title'].values:
    distances, indices = neighbors.kneighbors(matrix.loc[[book]])
    
    for i in range(1, N_RECOMMENDS + 1):
      recomm = [matrix.index[indices.flatten()[i]], distances.flatten()[i]]
      list[1].append(recomm)
  
  return list

In [79]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Pleading Guilty', 35.298725189445584],
  ['Long After Midnight', 35.749125863438955],
  ['Jackdaws', 36.09709129556009],
  ['Eaters of the Dead', 36.3318042491699],
  ['Last Man Standing', 36.701498607005135]]]

In [80]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Surgeon', 61.28621378417825], ['Unspeakable', 61.5223536610881], ['The Perks of Being a Wallflower', 61.57921727336261], ['Gap Creek: The Story Of A Marriage', 61.6765757804371], ['The Weight of Water', 61.75759062657804]]]
You haven't passed yet. Keep trying!
